#### Version 1
- words < 3 consider with next sentence
- score : 0.130

#### Version 2 
- words < 5 consider with next sentence
- score : 0.141

#### Version 3 (plan)
- try with `?!.` splits only
- score

In [ ]:
# To use 
import os
import shutil
if os.path.exists('/kaggle/temp/wheels'):
    shutil.rmtree('/kaggle/temp/wheels')
if os.path.exists('/kaggle/working/wheels'):
    shutil.rmtree('/kaggle/working/wheels')
# !unzip -q ../input/offline-package-simpletransformer/wheels -d /kaggle/working/wheels 
!pip install \
    --requirement ../input/offline-package-simpletransformer/requirements.txt \
    --no-index \
    --find-links ../input/offline-package-simpletransformer/wheels/

In [ ]:
import pandas
train_df = pandas.read_csv('../input/feedback-prize-2021/train.csv')
train_df

## Words

In [ ]:
# def filter_words(words):
#     words = [word.lower() for word in words if word not in stop]
#     return words

# def calc_freq(words,label='',freq = {}):
#     for word in words:
#         if not word in freq:
#             freq[word] = {}
#         if not label in freq[word]:
#             freq[word][label] = 0
#         freq[word][label] += 1
#     return freq

In [ ]:
# train = train_df[['discourse_text','discourse_type']].copy()
# train.columns = ['text','labels']
# train

In [ ]:
# train['labels'].unique()
# i = 0
label_map = {'Lead': 0,
 'Position': 1,
 'Evidence': 2,
 'Claim': 3,
 'Concluding Statement': 4,
 'Counterclaim': 5,
 'Rebuttal': 6}

# for label in train['labels'].unique():
#     label_map[label] = i
#     i += 1
# label_map

In [ ]:
# for i in range(len(train)):
#     train.loc[i, 'labels'] = label_map[train.loc[i,'labels']]    
# train

In [ ]:
# from sklearn.utils.class_weight import compute_class_weight
# lbl = train['labels'].unique()
# print(lbl)
# cw = compute_class_weight('balanced', classes = lbl, y=train['labels'])
# cw

In [ ]:
# import numpy
# # shuffle
# train = train.reindex(numpy.random.permutation(train.index))
# # 129864
# train_data = train[:129864] # 10 %
# val_data = train[129864:]

In [ ]:
from simpletransformers.classification import ClassificationModel,ClassificationArgs
# import logging
# from tqdm.notebook import tqdm

# logging.basicConfig(level=logging.INFO)
# transformers_logger = logging.getLogger("transformers")
# transformers_logger.setLevel(logging.WARNING)
# train_args = {
#     "reprocess_input_data": True,
#     "overwrite_output_dir": True,
#     "use_cached_eval_features": True,
#     "output_dir": f"outputs/roberta",
#     "best_model_dir": f"outputs/roberta/best_model",
#     "evaluate_during_training": True,
#     "max_seq_length": 128,
#     "num_train_epochs": 3,
#     "evaluate_during_training_steps": 1000,
#     "save_model_every_epoch": False,
#     "save_eval_checkpoints": False,
#     "train_batch_size": 128,
#     "eval_batch_size": 64,
# }

# model = ClassificationModel(
#     'roberta',
#     'roberta-base',
#     num_labels=len(lbl),
#     weight=[2.21529132, 1.33687565, 0.45103684, 0.41055779, 1.52634474, 3.54362828, 4.75289041],
#     use_cuda=True,
#     args=train_args,
# )
# model.train_model(train_data, eval_df=val_data)
# result, model_outputs, wrong_predictions = model.eval_model(val_data)

In [ ]:
model = ClassificationModel('roberta','../input/feedbackroberta/outputs/roberta/best_model')
model

In [ ]:
import re
import os
import nltk
import pandas

def get_label(val):
    global label_map
#     print(label_map,val)
    for key in label_map:
        if label_map[key] == val:
            return key
    return 'Concluding Statement'

pred = {'id':[],'class':[],'predictionstring':[]}

def make_lines(lines):
    new_lines = []
    token_lengths = []
    i = 0
    temp_line = ''
    word_len = 0
    for line in lines:
        total_words = len(nltk.word_tokenize(line))
        if total_words > 5:
            new_lines.append((temp_line+line).strip())
            token_lengths.append(word_len+total_words)
            temp_line = ''
            word_len = 0
        else:
            temp_line += line
            word_len += total_words
    return new_lines,token_lengths
        
           
for filename in os.listdir('../input/feedback-prize-2021/test'):
    with open(f'../input/feedback-prize-2021/test/{filename}') as file:
        reader = file.read()
        lines = re.compile('\.|\?|!').split(reader)
        lines, total_words = make_lines(lines)
        word_num = 0
        predictions, raw_outputs = model.predict(lines)
        i=0
        idname = filename[:-4]
        for line,words in zip(lines,total_words):
            pred['id'].append(idname)
            pred['class'].append(get_label(predictions[i]))
            pred['predictionstring'].append(' '.join([str(j) for j in range(word_num+1, word_num+words+1)]))
            word_num += words
            i += 1

In [ ]:
submit=pandas.DataFrame(pred)
submit

## Freq Count

In [ ]:
display(submit['class'].value_counts())
submit.to_csv('submission.csv',index=False)